# Explore the Dataset
## Iowa Liquor Sales from Kaggle
### https://www.kaggle.com/residentmario/iowa-liquor-sales

To be able to ETL this file we need to get information about. This file is quite challenging since its ~3.5GB and +12 Millions rows to wrangle.

First we need to get informations about the data type of each columns. We can work on the first 20 rows to get insight then refine some observation.

In [1]:
# install and import some useful librairies
!conda install -y pandas
import pandas as pd
import time

Solving environment: done

# All requested packages already installed.



In [2]:
# set some pandas default values to display every information
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# import os
# notebook_path = os.path.abspath("Explore_dataset.ipynb")
df_sample = pd.read_csv('./dataset/dataset.csv', nrows=20)
print(df_sample.head(20))
print(df_sample.dtypes)

   Invoice/Item Number        Date  Store Number  \
0         S29198800001  11/20/2015          2191   
1         S29195400002  11/21/2015          2205   
2         S29050300001  11/16/2015          3549   
3         S28867700001  11/04/2015          2513   
4         S29050800001  11/17/2015          3942   
5         S28869200001  11/11/2015          3650   
6         S28865700001  11/09/2015          2538   
7         S28869500001  11/10/2015          3942   
8         S29339300091  11/30/2015          2662   
9         S29050900001  11/16/2015          4307   
10        S29049900001  11/17/2015          2661   
11        S28868200001  11/05/2015          2561   
12        S28869600001  11/09/2015          4114   
13        S28866900001  11/11/2015          3650   
14        S29050100001  11/19/2015          2806   
15        S29049600001  11/17/2015          2624   
16        S28868400001  11/04/2015          2572   
17        S29196300002  11/24/2015          2595   
18        S2

From this sample I can start to come to some conclusion.

1. I can identify the column's name easily and start guessing the data type of each columns. That will speed up the reading of the full dataset later.
2. The column 'Invoice/Item Number' seems to be a unique identifier that I can use for each rows. Change the name of the colum to be `id` (step 2b).
3. 'Date' is obviously a date format that I have to convert from `string/object` to `datetime`.
4. 'ZipCode' is automaticaly typed as `object` because despite most of the data look like a numeric value, as attended, we have row 9: `712-2`. I quick search on my favourite search engine [Qwant](www.qwant.com) shows me that it's an Area Code. I will then consider `NaN` a non numeric value in this column.
5. Since there is some errors to handle on the address columns, I will not take the values in `Store Location` for granted and use the [Geocoding Google API](https://developers.google.com/maps/documentation/geocoding/intro) to create a `Lat`, `Lon` columns with the right values in the ETL process.
6. `Category` column is set as a float but on the sample we can only see integer like number, I have to explore this to optimize the memory spended. I also note that there will be `NaN` to handle here.
7. For future calculation it's going to be easier if the currency value is treated as numeric. I need to get rid of the dollard sign and convert the value as cents to handle integer instead of float (Bank and fintech use this optimization to handle billions of transaction with less memory used).
8. We have the bottle volume as **ml** (mililiter) and we have total `Volume Sold` in both **liters** and **gallons**. I prefer to use the [SI](https://en.wikipedia.org/wiki/International_System_of_Units) units for more convinence. So I will get rid of the extra column in **gallon**'s unit and convert the **liter** unit to **ml** for the same reason as the currency column's values.

Let try to do this 8 steps (not necesseraly in this order) on this sample data. That will be useful for the final ETL in a wrangling step.

In [4]:
from numpy import uint32
# step 1
'''
- Based on the raw data when I open the csv I can prepare 
- the coulumns' name and theirs raw types.
- That's will be usefull when I will work with the full 
- dataset by speeding up the opening/reading/parsing process
'''

from pandas import UInt32Dtype
from numpy import uint16, float32

datatype = {"Invoice/Item Number": object,
            "Date": object,
            "Store Number": uint32,
            "Store Name": object,
            "Address": object,
            "City": object,
            "Zip Code": object,
            "Store Location": object,
            "County Number": UInt32Dtype(),
            "County": object,
            "Category": object,
            "Category Name": object,
            "Vendor Number": UInt32Dtype(),
            "Vendor Name": object,
            "Item Number": uint32,
            "Item Description": object,
            "Pack": uint32,
            "Bottle Volume (ml)": uint32,
            "State Bottle Cost": object,
            "State Bottle Retail": object,
            "Bottles Sold": uint32,
            "Sale (Dollars)": object,
            "Volume Sold (Liters)": float32,
            "Volume Sold (Gallons)": float32}

cols = list(datatype.keys())

df_sample2 = pd.read_csv('./dataset/dataset.csv', dtype=datatype, usecols=cols, nrows=20)

print(df_sample2.dtypes)

Invoice/Item Number       object
Date                      object
Store Number              uint32
Store Name                object
Address                   object
City                      object
Zip Code                  object
Store Location            object
County Number             UInt32
County                    object
Category                  object
Category Name             object
Vendor Number             UInt32
Vendor Name               object
Item Number               uint32
Item Description          object
Pack                      uint32
Bottle Volume (ml)        uint32
State Bottle Cost         object
State Bottle Retail       object
Bottles Sold              uint32
Sale (Dollars)            object
Volume Sold (Liters)     float32
Volume Sold (Gallons)    float32
dtype: object


In [5]:
# step 2
df_sample2.rename(columns = {'Invoice/Item Number':'id'}, inplace = True)
df_sample2.set_index("id", inplace=True)

print(df_sample2.head(20))
print(df_sample2.dtypes)

                    Date  Store Number                         Store Name  \
id                                                                          
S29198800001  11/20/2015          2191                     Keokuk Spirits   
S29195400002  11/21/2015          2205             Ding's Honk And Holler   
S29050300001  11/16/2015          3549               Quicker Liquor Store   
S28867700001  11/04/2015          2513   Hy-Vee Food Store #2 / Iowa City   
S29050800001  11/17/2015          3942                   Twin Town Liquor   
S28869200001  11/11/2015          3650         Spirits, Stogies and Stuff   
S28865700001  11/09/2015          2538    Hy-Vee Food Store #3 / Waterloo   
S28869500001  11/10/2015          3942                   Twin Town Liquor   
S29339300091  11/30/2015          2662  Hy-Vee Wine & Spirits / Muscatine   
S29050900001  11/16/2015          4307         Crossroads Wine and Liquor   
S29049900001  11/17/2015          2661        Hy-Vee Food Store / Sheldon   

In [6]:
# step 3
df_sample2['Date'] = pd.to_datetime(df_sample2['Date'], infer_datetime_format=True)
print(df_sample2['Date'].head(20))
print(df_sample2['Date'].dtypes)


id
S29198800001   2015-11-20
S29195400002   2015-11-21
S29050300001   2015-11-16
S28867700001   2015-11-04
S29050800001   2015-11-17
S28869200001   2015-11-11
S28865700001   2015-11-09
S28869500001   2015-11-10
S29339300091   2015-11-30
S29050900001   2015-11-16
S29049900001   2015-11-17
S28868200001   2015-11-05
S28869600001   2015-11-09
S28866900001   2015-11-11
S29050100001   2015-11-19
S29049600001   2015-11-17
S28868400001   2015-11-04
S29196300002   2015-11-24
S29134300126   2015-11-18
S28869000001   2015-11-10
Name: Date, dtype: datetime64[ns]
datetime64[ns]


In [7]:
# step 4
df_sample2['Zip Code'] = pd.to_numeric(df_sample2['Zip Code'], errors='coerce', downcast='float')
print(df_sample2['Zip Code'].head(20))
print(df_sample2['Zip Code'].dtypes)

id
S29198800001    52632.0
S29195400002    51632.0
S29050300001    52627.0
S28867700001    52240.0
S29050800001    52342.0
S28869200001    51025.0
S28865700001    50702.0
S28869500001    52342.0
S29339300091    52761.0
S29050900001        NaN
S29049900001    51201.0
S28868200001    50321.0
S28869600001    50022.0
S28866900001    51025.0
S29050100001    52732.0
S29049600001    52002.0
S28868400001    50613.0
S29196300002    51442.0
S29134300126    51040.0
S28869000001    50263.0
Name: Zip Code, dtype: float32
float32


In [8]:
# step 5 - Just dropping the column 'Store Location', I will geocode in the ETL
df_sample2 = df_sample2.drop(columns=['Store Location'])

In [9]:
# step 6 - done on the sample but needed to verify on the full dataset later - float 32 seems to be enough to handle NaN
df_sample2['Category'] = pd.to_numeric(df_sample2['Category'], errors='coerce', downcast='float')
print(df_sample2['Category'].head(20))
print(df_sample2['Category'].dtypes)

id
S29198800001          NaN
S29195400002          NaN
S29050300001          NaN
S28867700001          NaN
S29050800001          NaN
S28869200001          NaN
S28865700001    1701100.0
S28869500001          NaN
S29339300091    1701100.0
S29050900001          NaN
S29049900001          NaN
S28868200001          NaN
S28869600001          NaN
S28866900001    1701100.0
S29050100001          NaN
S29049600001          NaN
S28868400001          NaN
S29196300002          NaN
S29134300126    1081200.0
S28869000001          NaN
Name: Category, dtype: float32
float32


In [10]:
def converting_str_currency_to_int(col):
    col = col.str.slice(1)
    col = pd.to_numeric(col, downcast='float')
    col = col * 100
    col = col.round(0)
    return pd.to_numeric(col, downcast='unsigned')
        

In [11]:
# step 7.1 - need to build function in the ETL to factorize this.
# df_sample2['State Bottle Cost'] = df_sample2['State Bottle Cost'].str.slice(1)
# df_sample2['State Bottle Cost'] = pd.to_numeric(df_sample2['State Bottle Cost'], downcast='float')
# df_sample2['State Bottle Cost'] = df_sample2['State Bottle Cost'] * 100
# df_sample2['State Bottle Cost'] = df_sample2['State Bottle Cost'].round(0)
# df_sample2['State Bottle Cost'] = pd.to_numeric(df_sample2['State Bottle Cost'], downcast='unsigned')
df_sample2['State Bottle Cost'] = converting_str_currency_to_int(df_sample2['State Bottle Cost'])

print(df_sample2['State Bottle Cost'].head(20))
print(df_sample2['State Bottle Cost'].dtypes)

id
S29198800001    1809
S29195400002    1809
S29050300001     640
S28867700001    3555
S29050800001     640
S28869200001    3555
S28865700001    1162
S28869500001    3555
S29339300091    1958
S29050900001     640
S29049900001     640
S28868200001    3555
S28869600001    3555
S28866900001    1162
S29050100001     640
S29049600001     640
S28868400001    3555
S29196300002    1809
S29134300126    9900
S28869000001    3555
Name: State Bottle Cost, dtype: uint16
uint16


In [12]:
# step 7.2
df_sample2['State Bottle Retail'] = converting_str_currency_to_int(df_sample2['State Bottle Retail'])

print(df_sample2['State Bottle Retail'].head(20))
print(df_sample2['State Bottle Retail'].dtypes)

id
S29198800001     2714
S29195400002     2714
S29050300001      960
S28867700001     5334
S29050800001      960
S28869200001     5334
S28865700001     1743
S28869500001     5334
S29339300091     2937
S29050900001      960
S29049900001      960
S28868200001     5334
S28869600001     5334
S28866900001     1743
S29050100001      960
S29049600001      960
S28868400001     5334
S29196300002     2714
S29134300126    14850
S28869000001     5334
Name: State Bottle Retail, dtype: uint16
uint16


In [13]:
# step 7.3
df_sample2['Sale (Dollars)'] = converting_str_currency_to_int(df_sample2['Sale (Dollars)'])

print(df_sample2['Sale (Dollars)'].head(20))
print(df_sample2['Sale (Dollars)'].dtypes)

id
S29198800001    16284
S29195400002    32568
S29050300001     1920
S28867700001    16002
S29050800001     1920
S28869200001     5334
S28865700001    10458
S28869500001    10668
S29339300091    11748
S29050900001     1920
S29049900001    19200
S28868200001    10668
S28869600001    16002
S28866900001     1743
S29050100001    19200
S29049600001    19200
S28868400001    16002
S29196300002    16284
S29134300126    14850
S28869000001    10668
Name: Sale (Dollars), dtype: uint16
uint16


In [14]:
# step 8
df_sample2['Volume Sold (Liters)'] = df_sample2['Volume Sold (Liters)'] * 100
df_sample2['Volume Sold (Liters)'] = df_sample2['Volume Sold (Liters)'].round(0)
df_sample2['Volume Sold (Liters)'] = pd.to_numeric(df_sample2['Volume Sold (Liters)'], downcast='unsigned')
df_sample2.rename(columns = {'Volume Sold (Liters)':'Volume Sold'}, inplace = True)

df_sample2 = df_sample2.drop(columns=['Volume Sold (Gallons)'])

print(df_sample2['Volume Sold'].head(20))
print(df_sample2['Volume Sold'].dtypes)

id
S29198800001    450
S29195400002    900
S29050300001     30
S28867700001    525
S29050800001     30
S28869200001    175
S28865700001    900
S28869500001    350
S29339300091    300
S29050900001     30
S29049900001    300
S28868200001    350
S28869600001    525
S28866900001    150
S29050100001    300
S29049600001    300
S28868400001    525
S29196300002    450
S29134300126    600
S28869000001    350
Name: Volume Sold, dtype: uint16
uint16


In [15]:
# final view of the data type
df_sample2.columns = df_sample2.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
print(df_sample2.head(20))
print(df_sample2.dtypes)

                   date  store_number                         store_name  \
id                                                                         
S29198800001 2015-11-20          2191                     Keokuk Spirits   
S29195400002 2015-11-21          2205             Ding's Honk And Holler   
S29050300001 2015-11-16          3549               Quicker Liquor Store   
S28867700001 2015-11-04          2513   Hy-Vee Food Store #2 / Iowa City   
S29050800001 2015-11-17          3942                   Twin Town Liquor   
S28869200001 2015-11-11          3650         Spirits, Stogies and Stuff   
S28865700001 2015-11-09          2538    Hy-Vee Food Store #3 / Waterloo   
S28869500001 2015-11-10          3942                   Twin Town Liquor   
S29339300091 2015-11-30          2662  Hy-Vee Wine & Spirits / Muscatine   
S29050900001 2015-11-16          4307         Crossroads Wine and Liquor   
S29049900001 2015-11-17          2661        Hy-Vee Food Store / Sheldon   
S28868200001

We now have all the needed steps to have a **ready-to-stage** dataframe into a Database.

For this use case I will stage the data into [Postgres](https://www.postgresql.org/) available with [PgAdmin4](https://www.pgadmin.org/) thank to this docker-compose on http://localhost:80.

Because there is only 3.5GB of data and because I want this project to be self sustainable without settings anything else that isn't available in those docker containers, those are the reasons why I will stick with postgres for the purpose of this Capstone project.

Of course in a real Big Data project I would prefer to use [AWS Redshift](https://aws.amazon.com/redshift/) or [Azure Datawarehouse](https://azure.microsoft.com/en-us/services/sql-data-warehouse/) and [Spark](https://spark.apache.org/).

All the ETL workflow will be done with [Apache Airflow](https://airflow.apache.org/), where the UI is available on http://localhost:8080

### Star Schema

With this raw data I will build a start schema to make the data easily queriable for data analysis.

![star_schema](start_schema.png)

The rest of the process will be done with Airflow the build a DAG :

![airflow_dag](dag.png)